<a id='intro'></a>
## Indice
* [**1. Introduction**](#gammapy)

* [**2. Setup**](#setup)
* [**3. Data Access and Selection (DL3)**](#3.0)
    * [**3.1 Gammapy Catalogs**](#3.1)
    * [**3.2 LHASSO Data Published in the Nature**](#3.2)

Notations

We use the following variable names :

....


|Variable| Dataset attribute name |Definition | 
|:-|  :-|:- |
|src|source | ... |
|fp|flux points | ... |



<a id='setup'></a>
🔝 [Back to Top](#intro)<br>
## 2. Setup 

In [1]:
from gammapy.utils.check import check_tutorials_setup
from gammapy.visualization.utils import plot_contour_line

check_tutorials_setup()


System:

	python_executable      : /home/gamma/anaconda3/envs/gammapy-1.0.1/bin/python 
	python_version         : 3.9.16     
	machine                : x86_64     
	system                 : Linux      


Gammapy package:

	version                : 1.0.1      
	path                   : /home/gamma/anaconda3/envs/gammapy-1.0.1/lib/python3.9/site-packages/gammapy 


Other packages:

	numpy                  : 1.24.2     
	scipy                  : 1.10.1     
	astropy                : 5.1.1      
	regions                : 0.7        
	click                  : 8.1.3      
	yaml                   : 6.0        
	IPython                : 8.7.0      
	jupyterlab             : 3.5.3      
	matplotlib             : 3.7.1      
	pandas                 : 1.5.2      
	healpy                 : 1.16.1     
	iminuit                : 2.20.0     
	sherpa                 : 4.15.0     
	naima                  : 0.10.0     
	emcee                  : 3.1.3      
	corner                 : 2.2.1      


Gammap

In [2]:
# !pip install dataframe-image

In [3]:
# !pip install tabulate

In [4]:
# # to install the latest commit (from GitHub)
# !pip install git+https://github.com/garrettj403/SciencePlots

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
import scienceplots

ModuleNotFoundError: No module named 'scienceplots'

In [ ]:
# Making plots actually look nice
plt.style.use(['science', 'notebook', 'grid'])

In [ ]:
# plt.style.available

In [ ]:
from gammapy.datasets import FluxPointsDataset, Datasets
from gammapy.catalog import CATALOG_REGISTRY

In [ ]:
import os
import sys
import importlib

modules_path = '../modules'

module_path = os.path.abspath(f'{modules_path}/spectral_models')
if module_path not in sys.path:
    sys.path.append(module_path)

import spectral_models
importlib.reload(spectral_models)
from spectral_models import pwl_model, get_source_data

module_path = os.path.abspath(f'{modules_path}/utilities')
if module_path not in sys.path:
    sys.path.append(module_path)

import utilities
importlib.reload(utilities)
from utilities import mkdir_sub_directory

In [ ]:
from astropy import units as u

from gammapy.modeling import Fit
from gammapy.modeling.models import (
    Models,
    SuperExpCutoffPowerLaw4FGLDR3SpectralModel,
    ExpCutoffPowerLawSpectralModel,
    LogParabolaSpectralModel,
    PowerLawSpectralModel,
    SkyModel,
)

In [ ]:
def source_identification(source_index = None):
    '''
    To write Docstring!!!!
    '''
    try: 
        
        global PeVatron_name, PeVatron_txt 
        
        PeVatron_name = df_fp["PeVatron_name"][source_index]
        PeVatron_txt = df_fp["PeVatron_txt"][source_index]
        
        global counterpart_name, counterpart_txt
        
        counterpart_name = df_fp["counterpart_name"][source_index]
        counterpart_txt = df_fp["counterpart_txt"][source_index] 
    
    except:
        print("Oops!  That was no valid index.  Try again...")

In [ ]:
# for i, dict_key in enumerate(list(dict_TeV_counterparts.keys())):
#     print(dict_key)

In [ ]:
# source = catalog_hgps['HESS J1826-130']


In [ ]:
# source.data.

In [ ]:
# PeVatron_name ='LHAASO J1825-1326'
# dict_TeV_counterparts[PeVatron_name]

In [ ]:
# import pandas library as pd

import pandas as pd 
from gammapy.datasets import FluxPointsDataset
from gammapy.catalog import CATALOG_REGISTRY
import os
import sys

format_csv = ".csv"
format_fits = ".fits"

sed_type="e2dnde"
# sed_type="dnde"

df_columns=[]
datasets_fp = []
sources_catalogs = []
  
# create an Empty DataFrame object
df = pd.DataFrame()


def getting_source_data(dict_TeV_counterparts = None, catalogs_tags = None):
    '''
    To write Docstring!!!!
    '''
    
#     LHAASO_range = range(len())
    for PeVatron_name in list(dict_TeV_counterparts.keys()):
#     for i in LHAASO_range:

#         PeVatron_name = dict_key[i]
        PeVatron_txt = PeVatron_name.replace(" ", "")
        
#         source_range = range(len())
        for counterpart_name in dict_TeV_counterparts[PeVatron_name]:
            
            counterpart_txt = counterpart_name.replace(" ", "")

            catalog_source = []

            catalog_fp = []
            
            source_catalog = []

            dataset_fp = []

#             counterpart_name = dict_TeV_counterparts[PeVatron_name][j]
            
#             catalog_range = range(len(catalogs_tags))
#             for k in catalog_range: 
            for catalog_tag in catalogs_tags:    
#                 catalog_tag = catalogs_tags[k]

                catalog = CATALOG_REGISTRY.get_cls(catalog_tag)()
                
                try:
                    
                    src = catalog[counterpart_name]
                    
                    catalog_source.append(catalog_tag)
                    
                    ds_name = f"{counterpart_name}: {catalog_tag}"
                    ds = FluxPointsDataset(
                        models = src.sky_model(),
                        data=src.flux_points, 
                        name=  ds_name     
                    )
                    
                    source_catalog.append(src)
                    dataset_fp.append(ds)
                    catalog_fp.append(catalog_tag)

                    table = ds.data.to_table(
                        sed_type = sed_type, 
                        formatted = True
                    )
                    
                    # Setting the units

                    
                    # Writes the flux points table in the fits format
                    file_name = f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}{format_fits}'
                    path_os = os.path.abspath(
                        os.path.join(
                            f"{path_flux_points_tables}/{file_name}"
                        )
                    )      
                    
                    if path_os not in sys.path:
                        sys.path.append(path_os)

                    table.write(
                        f"{path_os}",
                        format = 'fits', 
                        overwrite = True
                    )
                    
                    # Writes the flux points table in the csv format
                    file_name = f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}{format_csv}'
                    path_os = os.path.abspath(
                        os.path.join(
                            f"{path_flux_points_tables}/{file_name}"
                        )
                    )
                     
                    if path_os not in sys.path:
                        sys.path.append(path_os)

                    table.write(
                        f"{path_os}",
                        format = 'ascii.ecsv', 
                        overwrite = True
                    )   
                    
                except:
                    pass

                df_columns_source = [PeVatron_name, PeVatron_txt, counterpart_name, counterpart_txt, catalog_source, catalog_fp]
            df_columns.append(df_columns_source)
            datasets_fp.append(dataset_fp)
            sources_catalogs.append(source_catalog)

    df = pd.DataFrame(df_columns, columns = ['PeVatron_name', "PeVatron_txt",'counterpart_name', "counterpart_txt", 'catalog', 'flux_points']) 
    df_fp = df[df["flux_points"].str.len() != 0]
    df_fp = df_fp.reset_index(drop = True)
    df_fp.index.name = 'counterpart_index'
    
    file_name = "data_frame_flux_points.csv"
    path_file = f"{os.getcwd()}/{file_name}"
    df_fp.to_csv(path_file, index = "True" )
    
    df_no_fp = df[df["flux_points"].str.len() == 0]
    df_no_fp = df_no_fp.reset_index(drop = True)
    df_no_fp.index.name = df_fp.index.name
    
    file_name = "data_frame_no_flux_points.csv"
    path_file = f"{os.getcwd()}/{file_name}"
    df_no_fp.to_csv(path_file, index = "True")
    print(f"Sources without flux points table: {list(df_no_fp['counterpart_name'])}")
    
    sources_catalogs[:] = list(filter(None, sources_catalogs))
    datasets_fp[:] = list(filter(None, datasets_fp))
    
    return df_fp, datasets_fp, sources_catalogs

In [ ]:
# from pandas import read_csv

# myData = read_csv(f"{os.getcwd()}/data_frame_flux_points.csv", index_col = None)
# myData

In [ ]:
#to get the current working directory
directory = os.getcwd()

print(directory)

In [ ]:
import os

In [ ]:
path_DL3 = os.getcwd()
os.chdir('../')
path_LHAASO = os.getcwd()
os.chdir(path_DL3)

file_name = "paths.dat"
path_file = f"{path_LHAASO}/{file_name}"
    
with open(path_file, "w") as my_file: 
    my_file.write(f"{path_LHAASO} \n")
    my_file.write(f"{path_file} \n")
    my_file.write(f"{path_DL3} \n")

In [ ]:
# path_file

In [ ]:
# f"{os.getcwd()}/data_frame_no_flux_points.csv"

<a id='3.0'></a>
🔝 [Back to Top](#intro)<br>
## 3. Data Access and Selection (DL3) 

<a id='3.1'></a>
🔝 [Back to Top](#intro)<br>
#### 3.1 Gammapy Catalogs
Loading the catalogs and sources to be studied

In [ ]:
print(CATALOG_REGISTRY)

In [ ]:
catalogs_tags = ["gamma-cat", "hgps", "2hwc", "3fgl", "4fgl", "2fhl", "3fhl", "3hwc"] 

In [ ]:
catalogs_cls = []

In [ ]:
# Loading the source catalogs
catalog_gammacat = CATALOG_REGISTRY.get_cls("gamma-cat")()
catalogs_cls.append(catalog_gammacat)

catalog_hgps = CATALOG_REGISTRY.get_cls("hgps")()
catalogs_cls.append(catalog_hgps)

catalog_2hwc = CATALOG_REGISTRY.get_cls("2hwc")()
catalogs_cls.append(catalog_2hwc)

catalog_3fgl = CATALOG_REGISTRY.get_cls("3fgl")()
catalogs_cls.append(catalog_3fgl)

catalog_4fgl = CATALOG_REGISTRY.get_cls("4fgl")()
catalogs_cls.append(catalog_4fgl)

catalog_2fhl = CATALOG_REGISTRY.get_cls("2fhl")()
catalogs_cls.append(catalog_2fhl)

catalog_3fhl = CATALOG_REGISTRY.get_cls("3fhl")()
catalogs_cls.append(catalog_3fhl)

catalog_3hwc = CATALOG_REGISTRY.get_cls("3hwc")()
catalogs_cls.append(catalog_3hwc)

In [ ]:
for catalog in catalogs_cls:
    print(f"{catalog.tag}: {catalog.description}")

In [ ]:
# Creating a dictionary of the Potential TeV Counterparts (dict values) 
# of the three most significant (excluding Crab) LHASSO observations (dict keys)
# see https://www.nature.com/articles/s41586-021-03498-z
dict_TeV_counterparts = {
    "LHAASO J1825-1326": ["HESS J1825-137", "HESS J1826-130", "2HWC J1825-134"], 
    "LHAASO J1908+0621": ["MGRO J1908+06" , "HESS J1908+063", "ARGO J1907+0627", "VER J1907+062", "2HWC J1908+063"], 
    "LHAASO J2226+6057": ["VER J2227+608"  , "Boomerang"]
}

In [ ]:
# Creating the directories to save the flux points tables from Gammapy catalogs
dir_flux_points_tables = "flux_points_tables"

path_flux_points_tables = mkdir_sub_directory(
    dir_flux_points_tables
)

In [ ]:
# Getting the data of the sources from the catalogs of the Gammapy package
df_fp, datasets_fp, sources_catalogs = getting_source_data(dict_TeV_counterparts, catalogs_tags)

In [ ]:
df_fp

In [ ]:
datasets_fp[1]

In [ ]:
import numpy as np
import astropy.units as u

# %matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
from gammapy.catalog import CATALOG_REGISTRY

Source catalogs

We will show how to load source catalogs with Gammapy and explore the data using the following classes:

SourceCatalog, specifically SourceCatalog3FHL

Table

In [ ]:
counterpart_name= list(dict_TeV_counterparts.values())[0][0]

In [ ]:
print(df_fp [['PeVatron_name','counterpart_name','flux_points']].to_latex(index=False)) 

In [ ]:
from pandas import read_csv

df_no_pf = read_csv(f"{os.getcwd()}/data_frame_no_flux_points.csv", index_col = 'counterpart_index')
df_no_pf

In [ ]:
print(df_no_pf[['PeVatron_name','counterpart_name', 'catalog']].to_latex(index=False)) 

<a id='3.2'></a>
🔝 [Back to Top](#intro)<br>
#### 3.2 Data Published in the Nature

In [ ]:
directory_path = '../publishNature'
# format_dat = '.dat'

In [ ]:
list(df_fp['PeVatron_name'])

In [ ]:
# To get unique values from a list
set_res = set(list(df_fp['PeVatron_name'])) # To convert the input list to set
# print(set_res) 
PeVatrons_names = (list(set_res)) # To convert the set back to the list 
# PeVatrons_names.reverse()

In [ ]:
PeVatrons_names

In [ ]:
files_names = ['J1825_KM2A_201209.dat', 'J2228_KM2A_201209.dat', 'J1908_KM2A_201209.dat']

In [ ]:
files_names

In [ ]:
from pathlib import Path
from astropy.table import Table

x_data, y_data, y_errp, y_errn = [], [], [], []
for i, file_name in enumerate(files_names):
    
    file_path = Path(f'{directory_path}/{file_name}') 
    table = Table.read(file_path,format='ascii', delimiter=' ', comment='#')
#     display(table)
    
    table['col1'] = table['col1']/1e12
    table.rename_column('col1', 'e_ref')
    table['e_ref'].unit = u.TeV

#     table['col5'] = table['col5']/1e12
#     table.rename_column('col5', 'e_min')
#     table['e_min'].unit = u.TeV

#     table['col6'] = table['col6']/1e12
#     table.rename_column('col6', 'e_max')
#     table['e_max'].unit = u.TeV

    table.rename_column('col2', 'e2dnde')
    table['e2dnde'].unit = u.Unit("erg cm-2 s-1")

    table.rename_column('col3', 'e2dnde_errp')
    table['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")

    table.rename_column('col4', 'e2dnde_errn')
    table['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

    table.meta["SED_TYPE"] = "e2dnde"
    table.meta["name"] = "table"
    
    print(PeVatrons_names[i])
    display(table)
    
    if i == 0:
        table_J1825 = Table(table, copy=True)
    elif i == 1:
        table_J1908 = Table(table, copy=True)
    else:
        table_J2226 = Table(table, copy=True)
   
    x_data.append(table['e_ref'])
    y_data.append(table['e2dnde'])
    y_errp.append(table['e2dnde_errp'])
    y_errn.append(table['e2dnde_errn'])

In [ ]:
table_J1825

In [ ]:
from gammapy.estimators import FluxPoints
FluxPoints.from_table(table = table_J1908, sed_type='e2dnde').to_table()

In [ ]:
flux_points_J1908 = FluxPoints.from_table(table = table_J1908, sed_type='e2dnde')
flux_points_J1908.plot()
plt.xlim(.1, 1000)
plt.ylim(1e-14, 1e-10)
plt.grid(which="both")

In [ ]:
ds_J1908 = FluxPointsDataset(
                        models = sky_model,
                        data=flux_points_J1908, 
                        name=  sky_model.name     
                    )

In [ ]:
spectral_model = LogParabolaSpectralModel(
    alpha=2.3,
    amplitude="1e-12 cm-2 s-1 TeV-1",
    reference=1 * u.TeV,
    beta=0.5,
)
# model.plot(energy_bounds)
# plt.grid(which="both")

sky_model = SkyModel(spectral_model=spectral_model, name=spectral_model.tag[0])



datasets = Datasets(ds_J1908)
datasets.models = sky_model

fitter = Fit()

result_fit = fitter.run(datasets=datasets)

plt.style.use(['default'])

ax = plt.subplot()
ax.yaxis.set_units(u.Unit("erg cm-2 s-1"))

kwargs = {"ax": ax, "sed_type": "e2dnde"}

for d in datasets:
    d.data.plot(label=d.name, **kwargs)

energy_bounds = (0.1 * u.TeV, 1000 * u.TeV)
spectral_model.plot(energy_bounds=energy_bounds, color="k", **kwargs)
ax.set_ylim(1e-16, 1e-8)
ax.set_xlim(energy_bounds)
ax.legend()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12,6), sharex=True, sharey=True)

ax = axes[0]
ax.set_ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")

ax.loglog(x_data[0], y_data[0], 'o', color='red', lw=2, ms=4)
ax.errorbar(x_data[0], y_data[0], yerr=[y_errn[0], y_errp[0]],fmt='.', color='red')
ax.text(0.04,0.1, PeVatrons_names[0], transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=13)
ax.grid(which="both")
ax.set_xlim(10, 1000)
ax.set_ylim(1e-14, 5e-11)


ax = axes[1]
ax.loglog(x_data[1], y_data[1], 'o', color='red', lw=2, ms=4)
ax.errorbar(x_data[1], y_data[1], yerr=[y_errn[1], y_errp[1]],fmt='.', color='red')
ax.text(0.04,0.1, PeVatrons_names[1], transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=13)
ax.grid(which="both")
ax.set_xlim(10, 1000)
ax.set_ylim(1e-14, 5e-11)

ax = axes[2]
ax.loglog(x_data[2], y_data[2], 'o', color='red', lw=2, ms=4)
ax.errorbar(x_data[2], y_data[2], yerr=[y_errn[2], y_errp[2]],fmt='.', color='red')
ax.text(0.04,0.1, PeVatrons_names[2], transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=13)
ax.set_xlim(10, 1000)
ax.set_ylim(1e-14, 5e-11)
ax.grid(which="both")
# ax.tick_params(axis='both', which='major', labelsize=10)

fig.text(0.5, -0.04, "Energy (TeV)", ha='center', size=20)

# fig.suptitle('Flux Points from LHAASO', y=1., fontsize=20)

plt.savefig('Cao2021-LHAASO_fp.png', bbox_inches='tight')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,6), sharex=True, sharey=True)

# ax = axes[0]
ax.set_ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")

ax.loglog(x_data[0], y_data[0], 'o', color='red', lw=2, ms=4)
ax.errorbar(x_data[0], y_data[0], yerr=[y_errn[0], y_errp[0]],fmt='.', color='red')
ax.text(0.04,0.1, PeVatrons_names[0], transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=13)
ax.grid(which="both")
ax.set_xlim(10, 1000)
ax.set_ylim(1e-14, 5e-11)


# ax = axes[1]
# ax.loglog(x_data[1], y_data[1], 'o', color='red', lw=2, ms=4)
# ax.errorbar(x_data[1], y_data[1], yerr=[y_errn[1], y_errp[1]],fmt='.', color='red')
# ax.text(0.04,0.1, PeVatrons_names[1], transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=13)
# ax.grid(which="both")
# ax.set_xlim(10, 1000)
# ax.set_ylim(1e-14, 5e-11)

# ax = axes[2]
# ax.loglog(x_data[2], y_data[2], 'o', color='red', lw=2, ms=4)
# ax.errorbar(x_data[2], y_data[2], yerr=[y_errn[2], y_errp[2]],fmt='.', color='red')
# ax.text(0.04,0.1, PeVatrons_names[2], transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=13)
# ax.set_xlim(10, 1000)
# ax.set_ylim(1e-14, 5e-11)
# ax.grid(which="both")
# ax.tick_params(axis='both', which='major', labelsize=10)

fig.text(0.5, -0.04, "Energy (TeV)", ha='center', size=20)

# fig.suptitle('Flux Points from LHAASO', y=1., fontsize=20)

plt.savefig('LHAASOJ1825-1326_plot.png', bbox_inches='tight')

plt.show()

#### HAWCJ1825

In [ ]:
with open('HAWCJ1825-134.csv', "r") as my_file: 
    df = my_file.read()

In [ ]:
print(df)

In [ ]:
table_HAWCJ1825_134 = Table.read('HAWCJ1825-134.csv',format='ascii', delimiter=',', comment='#')
table_HAWCJ1825_134.colnames

In [ ]:
table_HAWCJ1825_134.rename_column('x', 'e_ref')
table_HAWCJ1825_134['e_ref'].unit = u.TeV

table_HAWCJ1825_134['e2dnde'].unit = u.Unit("TeV cm-2 s-1")

In [ ]:
table_HAWCJ1825_134

In [ ]:
from gammapy.estimators import FluxPoints

In [ ]:
FluxPoints.from_table(table = table_HAWCJ1825_134, sed_type='e2dnde').plot()
plt.xlim(.1, 1000)
plt.ylim(1e-14, 1e-10)
plt.grid(which="both")

In [ ]:
yaxis_units="erg cm-2 s-1"
ax = plt.subplot()
ax.yaxis.set_units(u.Unit(yaxis_units))
ax.xaxis.set_units(u.Unit("TeV"))

flux_points = FluxPoints.from_table(table = table_HAWCJ1825_134, sed_type='e2dnde')
flux_points.plot(ax=ax)


plt.xlim(.1, 1000)
plt.ylim(1e-14, 1e-10)
plt.grid(which="both")
plt.savefig('uu.png', bbox_inches='tight')

In [ ]:
flux_points.to_table()

In [ ]:
table_HAWCJ1825_134["e2dnde"][:] = table_HAWCJ1825_134["e2dnde"].to(u.Unit("erg cm-2 s-1")) 

In [ ]:
table_HAWCJ1825_134

In [ ]:
flux_points = FluxPoints.from_table(table_HAWCJ1825_134, sed_type='e2dnde')

flux_points.to_table(formatted=True)

In [ ]:
dir_plot_spectrum = "SED_from_catalogs"

path_plot_spectrum = mkdir_sub_directory(
    dir_plot_spectrum
)

In [ ]:
def plot_SED():
    plt.style.use(['science', 'notebook', 'grid'])

    plt.figure(figsize=(8,6))

    ax = spectral_model.plot(energy_bounds, sed_type="e2dnde", color="red")
    ax = spectral_model.plot_error(energy_bounds, sed_type="e2dnde")
    flux_points.plot(
        ax=ax,
        color=colors_tag, 
        sed_type="e2dnde"
    )
    ax.text(0.02, .94, PeVatron_name, transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black'), size=13)

    ax.set_ylim(ylim)
    ax.set_xlim(energy_bounds)

    # plt.title(f"{PeVatron_name}: {counterpart_name}")
    plt.legend([spectral_model_tag, "Error", f"{counterpart_name} - {catalog_tag}"], facecolor='white', edgecolor='black',  loc='upper right', prop={'size': 8})
    plt.xlabel("Energy (TeV)")   
    plt.ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
    plt.savefig(file_path, bbox_inches='tight')
    
    plt.show()

In [ ]:
energy_bounds = [1e-5, 1e3] * u.TeV
ylim = [1e-16, 1e-6]

In [ ]:
# A color-blind/friendly color cycle for Matplotlib line plots. 
colors_tags =[
    '#377eb8', 
    '#ff7f00', 
    '#4daf4a',
    '#f781bf', 
    '#a65628', 
    '#984ea3',
    '#999999', 
    '#e41a1c', 
    '#dede00'
]

# colors_tags = ["b", "g", "r", "c", "m", "y", "k", "m"]

In [ ]:
catalog_index = 1

In [ ]:
df_fp

In [ ]:
df_fp['catalog'][3]

In [ ]:
source = sources_catalogs[3][0]

In [ ]:
print(source.info())

In [ ]:
# sources_catalogs[0][0].data['common_name', '']

In [ ]:
# catalog_3hwc["eHWC J1825-134"].data.keys()

In [ ]:
# type(df_fp["catalog"][source_index])

In [ ]:
# to select a source object index: [0-6]
source_index = 1
colors_tag = colors_tags[source_index]
source_identification(source_index)
# PeVatron_name, counterpart_name

# List of catalogs with flux points table of the source
# print(df_fp['catalog'][source_index])

k_range = range(len(datasets_fp[source_index]))
for k in k_range: 
            
    # to select a catalog:
    cat_index = k
    catalog_tag = df_fp["catalog"][source_index][cat_index]
    catalog = CATALOG_REGISTRY.get_cls(catalog_tag)()
    # print(catalog)

    source  = catalog[counterpart_name]
    # source.data

    flux_points = source.flux_points

    # Gammapy sky model object:
    sky_model = source.sky_model()

    # Gammapy spatial model object:
    spatial_model = source.spatial_model

    # Gammapy spectral model object:
    spectral_model = source.spectral_model()
    spectral_model_tag = spectral_model.tag[0]
    spectral_model_tag_short = spectral_model.tag[0]

    file_name = f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}_{spectral_model_tag_short}.png'
    file_path = path_plot_spectrum / file_name 
    
    plot_SED()
    
    print(f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}_{spectral_model_tag_short}:')
    
    print(spectral_model.parameters.to_table())
    

In [ ]:
# Creating the directories to save the flux points figures
# dir_analysis = "analysis"
dir_flux_points_figures = "flux_points_figures"

path_flux_points_figures_analysis = mkdir_sub_directory(
#     dir_analysis, 
    dir_flux_points_figures
)

In [ ]:
def plotting_SED_flux_pointing(datasets = None, spectral_model = None, sed_type = "e2dnde", yaxis_units="TeV cm-2 s-1", energy_bounds= [1e-5, 1e2] * u.TeV, ylim = [1e-13, 1e-9]):
    '''Returns a spectral energy distribution
    
    >>> plotting_SED_flux_pointing(spec_model, yaxis_units, sed_type, reference)
    '''
    
    ax = plt.subplot()
#     ax.grid(which="both")
    ax.yaxis.set_units(u.Unit(yaxis_units))
    ax.xaxis.set_units(u.Unit("TeV"))

    #label = f"{sky_model}"
    
    kwargs = {
        "ax": ax, 
        "sed_type": sed_type
    }    
    
    
#     plot_kwargs = {
#     "energy_bounds": [0.1, 30] * u.TeV,
#     "sed_type": "e2dnde",
#     "yunits": u.Unit("erg cm-2 s-1"),
#     "ax": ax,
# }
    
    PeVatron_name = PeVatrons_names[table_index]
    kwargs_PeV = {
        'label': PeVatron_name
    }
    ax.plot(x_data[table_index], y_data[table_index], 'o', color='red', lw=2, ms=4)
    ax.errorbar(x_data[table_index], y_data[table_index], yerr=[y_errn[table_index], y_errp[table_index]],fmt='.', color='red', **kwargs_PeV)


#     ax.loglog(J1825['e_ref'],J1825['e2dnde'], 'o', color='red', lw=2, ms=4)
#     ax.errorbar(J1825['e_ref'],J1825['e2dnde'], yerr=[J1825['e2dnde_errn'], J1825['e2dnde_errp']],fmt='.', color='red', label=PeVatron_name)

    color_tag = 0
    for d in datasets:
        d.data.plot(label=d.name, color=colors_tags[color_tag], **kwargs)
        color_tag += 1
    try:   

        kwargs_fit = {"label":sky_model.name}
        spectral_model.plot(energy_bounds=energy_bounds, color="k", **kwargs, **kwargs_fit)
        spectral_model.plot_error(energy_bounds=energy_bounds, **kwargs)
        
        if combined_ds == False:
            file = path_flux_points_figures_analysis / f'{PeVatron_txt}_{counterpart_txt}_{sky_model.name}.png'
        else:
            file = path_flux_points_figures_analysis / f'{PeVatron_txt}_{sky_model.name}.png'
            
    except:
        
        if combined_ds == False:
            file = path_flux_points_figures_analysis / f'{PeVatron_txt}_{counterpart_txt}.png'
        else:
            file = path_flux_points_figures_analysis / f'{PeVatron_txt}.png'
    

    
    ax.set_ylim(ylim)
    ax.set_xlim(energy_bounds)
    ax.legend(facecolor='white', edgecolor='white',  loc='upper right', prop={'size': 10})
    
#     ax.text(0.02, .94, PeVatron_name, transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='white'), size=15)
    
        
#     plt.title(f"{PeVatron_name}: {counterpart_name}")
    
    plt.xlabel("Energy (TeV)")   
    plt.ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
    
    plt.savefig(file, bbox_inches='tight')
    plt.show()
    
    return 

In [ ]:
import scienceplots
plt.style.use(['science', 'notebook', 'grid'])

# to select a source object index: [0-6]
source_index = 0
table_index = 0
source_identification(source_index)

ds_src = datasets_fp[source_index]

datasets = Datasets(ds_src)

energy_bounds = [1e-5, 1e3] * u.TeV
ylim = [1e-14, 1e-7]

combined_ds = False
plotting_SED_flux_pointing(
    datasets = datasets,
    sed_type = sed_type, 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim)

## 

In [ ]:
# to select a source object index: [0-6]
source_index = 0
table_index = 0
source_identification(source_index)

ds_src = datasets_fp[source_index]

datasets = Datasets(ds_src)

energy_bounds = [1e-5, 1e3] * u.TeV
ylim = [1e-14, 1e-7]

combined_ds = False
plotting_SED_flux_pointing(
    datasets = datasets,
    sed_type = sed_type, 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim)

In [ ]:
import scienceplots
plt.style.use(['science', 'notebook', 'grid'])

# to select a source object index: [0-6]
source_index = 6

source_identification(source_index)

PeVatron_name

In [ ]:
ds_src_combined = list(datasets_fp[source_index])

In [ ]:
ds_src_combined

In [ ]:
for item in datasets_fp[5]:
    ds_src_combined.append(item)

In [ ]:
ds_src_combined

In [ ]:
ds_src_combined_3 = list(ds_src_combined)

In [ ]:
ds_src_combined_2 = list(ds_src_combined)

In [ ]:
ds_src_combined_1 = list(ds_src_combined)

In [ ]:
datasets_combined = Datasets(ds_src_combined_3)

# energy_bounds = [1e-5, 1e2] * u.TeV
ylim = [1e-16, 1e-6]

table_index = 1
combined_ds = True
plotting_SED_flux_pointing(
    datasets = datasets_combined,
    sed_type = sed_type, 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim)

## Data Reduction (DL3toDL4)

In [ ]:
import numpy as np
# from astropy import units as u
from astropy.table import Table
from gammapy.estimators import FluxPoints
from gammapy.utils.scripts import make_path

# format_csv = ".csv"
format_fits = ".fits"

sed_type="e2dnde"
# sed_type="dnde"

datasets_fp_cut = []
def flux_points_cut_energy(df_fp = None, datasets_fp = None, e_ref_min=None):
    '''To write Docstring!!!!'''
    
    source_range = range(len(df_fp))
    for source_index in source_range:
        dataset_fp = []
        
        source_identification(source_index)
        
        k_range = range(len(datasets_fp[source_index]))
        for k in k_range: 
            
            catalog_tag = df_fp["catalog"][source_index][k]

            ds_name = f"{counterpart_name}: {catalog_tag}"
            flux_points = datasets_fp[source_index][k].data

            mask_energy = np.zeros(len(flux_points.to_table()), dtype=bool)

            for m, e_ref in enumerate(flux_points.energy_ref):
                if e_ref > e_ref_min:
                    mask_energy[m] = True
            print(ds_name, counterpart_txt)  
            
            try:
                flux_points_mask = flux_points.to_table()[mask_energy]
                flux_points_energy = FluxPoints.from_table(flux_points_mask)

                ds = FluxPointsDataset(
                                    data=flux_points_energy, 
                                    name=ds_name
                                )

                table = ds.data.to_table(
                                    sed_type = sed_type, 
                                    formatted = True
                                )
                dataset_fp.append(ds)

                file_name = f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}_e_ref_min_{e_ref_min_name}{format_fits}'
                path_os = os.path.abspath(
                    os.path.join(
                        f"{path_flux_points_tables_analysis}/{file_name}"
                    )
                )

                if path_os not in sys.path:
                    sys.path.append(path_os)

                table.write(f"{path_os}",format='fits', overwrite=True)

            except:
                print(f"{counterpart_name} out")    
            
        datasets_fp_cut.append(dataset_fp)
    return datasets_fp_cut

### Cutting energy

In [ ]:
# Defining the minimum value of the reference energy  
e_ref_min = 100 * u.Unit("GeV")
e_ref_min_name = str(e_ref_min).replace(" ", "")

In [ ]:
# Creating the directories to save the tables
dir_analysis = "analysis"
dir_flux_points_tables = f"flux_points_table_e_ref_min_{e_ref_min_name}"

path_catalogs_data, path_flux_points_tables_analysis = mkdir_sub_directory(
    dir_analysis, 
    dir_flux_points_tables
)

In [ ]:
datasets_fp_cut = flux_points_cut_energy(df_fp = df_fp, datasets_fp = datasets_fp, e_ref_min=e_ref_min)

In [ ]:
# Creating the directories to save the tables
dir_analysis = "analysis"
dir_flux_points_figures = f"flux_points_figures_e_ref_min_{e_ref_min_name}"

path_analysis, path_flux_points_figures_analysis = mkdir_sub_directory(
    dir_analysis, 
    dir_flux_points_figures
)

In [ ]:
# to select a source object index: [0-6]
source_index = 6

source_identification(source_index)

PeVatron_name

In [ ]:
ds_src_combined_cut = list(datasets_fp_cut[source_index])

In [ ]:
ds_src_combined_cut

In [ ]:
for item in datasets_fp_cut[5]:
    ds_src_combined_cut.append(item)

In [ ]:
ds_src_combined_cut

In [ ]:
ds_src_combined_cut_3 = list(ds_src_combined_cut)

In [ ]:
ds_src_combined_cut_2 = list(ds_src_combined_cut)

In [ ]:
ds_src_combined_cut_1 = list(ds_src_combined_cut)

In [ ]:
datasets_combined = Datasets(ds_src_combined_cut_3)

energy_bounds = [1e-1, 1e2] * u.TeV
# ylim = [1e-16, 1e-6]

combined_ds = True
plotting_SED_flux_pointing(
    datasets = datasets_combined,
    sed_type = sed_type, 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim)

### SED cutting energy

In [ ]:
# Creating the directories to save the tables
dir_analysis = "analysis"
dir_flux_points_figures = f"SED_figures_e_ref_min_{e_ref_min_name}"

path_analysis, path_flux_points_figures_analysis = mkdir_sub_directory(
    dir_analysis, 
    dir_flux_points_figures
)

In [ ]:
print(sky_model.parameters.to_table())

In [ ]:
spectral_model.parameters.to_table()

In [ ]:
# to select a source object index: [0-6]
table_index = 0
source_index = 0
cat_index = 2

# source = sources_catalogs[source_index][0]
source_identification(source_index)

catalog_tag = df_fp["catalog"][source_index][cat_index]

ds_src = datasets_fp[source_index]
# ds_src = ds_src_combined_cut_1

spectral_model = sources_catalogs[source_index][cat_index].spectral_model()
# spectral_model = PowerLawSpectralModel(
#     index=2,
#     amplitude="1e-10 TeV-1 cm-2 s-1",
#     reference=1 * u.TeV,
# )

name = f"{spectral_model.tag[0]}_{df_fp['catalog'][source_index][cat_index]}"
    
    
    
sky_model = SkyModel(spectral_model=spectral_model, name=name)

datasets = Datasets(ds_src)
datasets.models = sky_model

fitter = Fit()

result_fit = fitter.run(datasets=datasets)

energy_bounds = [1e-5, 1e3] * u.TeV
ylim = [1e-16, 1e-5]

plotting_SED_flux_pointing(
    datasets = datasets,
    spectral_model=spectral_model, 
    sed_type = "e2dnde", 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim
)

https://docs.gammapy.org/dev/tutorials/api/fitting.html

https://cxc.cfa.harvard.edu/sherpa/methods/index.html

In [ ]:
print(fit.minuit)

In [ ]:
display(result_fit.models.to_parameters_table())


In [ ]:
result_fit.models.covariance.plot_correlation()

# The covariance information is also propagated to the individual models
# Therefore, one can also get the error on a specific parameter by directly
# accessing the `~gammapy.modeling.Parameter.error` attribute:
#

print(sky_model.spectral_model.alpha.error)

In [ ]:
# to select a source object index: [0-6]
table_index = 1
source_index = 6
cat_index = 0

# source = sources_catalogs[source_index][0]
source_identification(source_index)

catalog_tag = df_fp["catalog"][source_index][cat_index]

ds_src = datasets_fp[source_index]
# ds_src = ds_src_combined_cut_1

spectral_model1 = sources_catalogs[source_index][cat_index].spectral_model()
# spectral_model = PowerLawSpectralModel(
#     index=2,
#     amplitude="1e-10 TeV-1 cm-2 s-1",
#     reference=1 * u.TeV,
# )





ecpwl = ExpCutoffPowerLawSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    index=2,
    lambda_=0.1 * u.Unit("TeV-1"),
    reference=0.01 * u.TeV,
)

lp1 = LogParabolaSpectralModel(
    alpha=1.56,
    amplitude="1e-15 cm-2 s-1 TeV-1",
    reference=50 * u.TeV,
    beta=0.88,
)


lp2 = LogParabolaSpectralModel(
    alpha=1.56,
    amplitude="1e-15 cm-2 s-1 erg-1",
    reference=5 * u.TeV,
    beta=0.88,
)

lp3 = LogParabolaSpectralModel(
    alpha=1.56,
    amplitude="1e-15 cm-2 s-1 erg-1",
    reference=50 * u.TeV,
    beta=0.88,
)


spectral_model = CompoundSpectralModel(spectral_model1, lp3, operator.add)



name = f"{spectral_model.tag[0]}_{df_fp['catalog'][source_index][cat_index]}"
    
sky_model = SkyModel(spectral_model=spectral_model, name=name)

datasets = Datasets(ds_src)
datasets.models = sky_model

fitter = Fit()

result_fit = fitter.run(datasets=datasets)

energy_bounds = [1e-5, 1e3] * u.TeV
ylim = [1e-18, 1e-5]

plotting_SED_flux_pointing(
    datasets = datasets,
    spectral_model=spectral_model, 
    sed_type = "e2dnde", 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim
)

In [ ]:
help(Fit())

In [ ]:
result_fit.parameters.to_table()

In [ ]:
counterpart_name

In [ ]:
J1825_flux_points = FluxPoints.from_table(table = table_J1825)

In [ ]:
J1825_ds = FluxPointsDataset(
    data = J1825_flux_points, 
    name = f"{PeVatron_name}"
)
datasets_fp_cut[source_index].append(J1825_ds)



In [ ]:
print(datasets_fp_cut[source_index][3])

#### AQUIII

In [ ]:
df_fp["PeVatron_name"]

In [ ]:
# to select a source object index: [0-6]
source_index = 6
table_index = 0
cat_index = 0
# source = sources_catalogs[source_index][0]
source_identification(source_index)

catalog_tag = df_fp["catalog"][source_index][cat_index]

ds_src = datasets_fp[source_index]
# ds_src = ds_src_combined_cut_3

spectral_model = sources_catalogs[source_index][cat_index].spectral_model()

# spectral_model = PowerLawSpectralModel(
#     index=2,
#     amplitude="1e-10 TeV-1 cm-2 s-1",
#     reference=1 * u.TeV,
# )

# spectral_model = LogParabolaSpectralModel(
#     alpha=0.92,
#     amplitude="1e-12 cm-2 s-1 erg-1",
#     reference=1 * u.TeV,
#     beta=1.19,
# )

name = f"{spectral_model.tag[0]}_{df_fp['catalog'][source_index][cat_index]}"
    
sky_model = SkyModel(spectral_model=spectral_model, name=name)

datasets = Datasets(ds_src)
datasets.models = sky_model

fitter = Fit()

result_fit = fitter.run(datasets=datasets)

energy_bounds = [1e-3, 1e5] * u.TeV
ylim = [1e-16, 1e-6]

plotting_SED_flux_pointing(
    datasets = datasets,
    spectral_model=spectral_model, 
    sed_type = "e2dnde", 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim
)

In [ ]:
print(result_fit.models.parameters.to_table())

In [ ]:
import operator
from astropy import units as u
import matplotlib.pyplot as plt
from gammapy.modeling.models import (
    CompoundSpectralModel,
    LogParabolaSpectralModel,
    Models,
    PowerLawSpectralModel,
    SkyModel,
)

energy_bounds = [0.1, 100] * u.TeV
pwl = PowerLawSpectralModel(
    index=2.0, amplitude="1e-12 cm-2 s-1 TeV-1", reference="1 TeV"
)
lp = LogParabolaSpectralModel(
    amplitude="1e-12 cm-2 s-1 TeV-1", reference="10 TeV", alpha=2.0, beta=1.0
)

model_add = CompoundSpectralModel(pwl, lp, operator.add)
model_add.plot(energy_bounds)
plt.grid(which="both")


In [ ]:
spectral_model.reference

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Generate dataset for the plot
# x = np.linspace(-10, 10, 1000)
# y = np.sin(x)

# # Plot the data
# plt.plot(x, y, color='red')

# # Use the rdbu colormap to color the background
# plt.imshow(np.outer(np.ones(10), np.arange(100)),
#            cmap='RdBu',
#            extent=(-10, 10, -1, 1),
#            alpha=0.2)

# # Add text to the plot using an rgba color
# plt.text(5, 0.5, 'Text in RGBA color', color=(0, 1, 0, 0.5), fontsize=16)

# # Show the plot
# plt.show()

In [ ]:
# to select a source object index: [0-6]
source_index = 0
source_identification(source_index)
PeVatron_name, counterpart_name

In [ ]:
# List of catalogs with flux points table of the source
df_fp['catalog'][source_index]

In [ ]:
# to select a catalog:
cat_index = 4
catalog_tag = df_fp["catalog"][source_index][cat_index]
catalog = CATALOG_REGISTRY.get_cls(catalog_tag)()
# print(catalog)

In [ ]:
source  = catalog[counterpart_name]
# source.data

In [ ]:
flux_points = source.flux_points
flux_points.to_table(sed_type="e2dnde", formatted=True)

In [ ]:
# Gammapy sky model object:
sky_model = source.sky_model()

# Gammapy spatial model object:
spatial_model = source.spatial_model

# Gammapy spectral model object:
spectral_model = source.spectral_model()
spectral_model_tag = spectral_model.tag[0]
spectral_model_tag_short = spectral_model.tag[0]

In [ ]:
# energy_bounds = [1e-5, 1e3] * u.TeV
# ylim = [1e-20, 1e-7]

In [ ]:
file_name = f'{PeVatron_txt}_{counterpart_txt}_{spectral_model_tag_short}.png'
file_path = path_plot_spectrum / file_name

In [ ]:
plt.style.use(['science', 'notebook', 'grid'])

plt.figure()

ax = spectral_model.plot(energy_bounds, sed_type="e2dnde")
ax = spectral_model.plot_error(energy_bounds, sed_type="e2dnde")
flux_points.plot(ax=ax, sed_type="e2dnde")
ax.text(0.08, 0.1, PeVatron_name, transform=ax.transAxes, fontsize = "x-large")

ax.set_ylim(ylim)
ax.set_xlim(energy_bounds)

# plt.title(f"{PeVatron_name}: {counterpart_name}")
plt.legend([spectral_model_tag, "Error", f"{counterpart_name} - {catalog_tag}"], loc='upper left', prop={'size': 10})
plt.savefig(file_path, bbox_inches='tight')
plt.show()

In [ ]:
file_name

In [ ]:
source_index = 1
source_identification(source_index)
catalog_tag = df_fp["catalog"][source_index][3]

file_name = f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}{format_fits}'
file_path = make_path(f"{path_flux_points_tables}/{file_name}")
flux_points = FluxPoints.read(file_path, sed_type=sed_type)
ds = FluxPointsDataset(
    data = flux_points, 
    name = f"{counterpart_name}: {catalog_tag}"
)
datasets = Datasets(ds)


# opts = dict(sed_type="e2dnde", yunits=u.Unit("TeV cm-2 s-1"))
energy_bounds = [1e-3, 1e3] * u.TeV
ylim = [1e-13, 1e-9]
plotting_SED_flux_pointing(
    datasets = datasets,
#     spectral_model=spectral_model, 
    sed_type = "e2dnde", 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim)


In [ ]:
# Defining the minimum value of the reference energy  
e_ref_min = 100 * u.Unit("GeV")
e_ref_min_name = str(e_ref_min).replace(" ", "")

In [ ]:
source_identification(source_index)
catalog_tag = df_fp["catalog"][source_index][3]

file_name = f'{PeVatron_txt}_{counterpart_txt}_{catalog_tag}_e_ref_min_{e_ref_min_name}{format_fits}'
file_path = make_path(f"{path_flux_points_tables_analysis}/{file_name}")
flux_points = FluxPoints.read(file_path, sed_type=sed_type)
ds = FluxPointsDataset(
    data = flux_points, 
    name = f"{counterpart_name}: {catalog_tag}"
)
datasets = Datasets(ds)


# opts = dict(sed_type="e2dnde", yunits=u.Unit("TeV cm-2 s-1"))
energy_bounds = [1e-3, 1e3] * u.TeV
ylim = [1e-13, 1e-9]
plotting_SED_flux_pointing(
    datasets = datasets,
#     spectral_model=spectral_model, 
    sed_type = "e2dnde", 
    yaxis_units="erg cm-2 s-1", 
    energy_bounds=energy_bounds, 
    ylim=ylim)


In [ ]:
with open('J1825_KM2A_201209.dat', "r") as my_file: 
    df = my_file.read()

## Table LHAASO

#### HAWCJ1825

In [ ]:
with open('HAWCJ1825-134.csv', "r") as my_file: 
    df = my_file.read()



In [ ]:
print(df)

In [ ]:
table_HAWCJ1825_134 = Table.read('HAWCJ1825-134.csv',format='ascii', delimiter=',', comment='#')
table_HAWCJ1825_134.colnames

In [ ]:
table_HAWCJ1825_134.rename_column('x', 'e_ref')
table_HAWCJ1825_134['e_ref'].unit = u.TeV


table_HAWCJ1825_134['e2dnde'].unit = u.Unit("TeV cm-2 s-1")

In [ ]:
table_HAWCJ1825_134

In [ ]:
FluxPoints.from_table(table = table_HAWCJ1825_134, sed_type='e2dnde').plot()
plt.xlim(.1, 1000)
plt.ylim(1e-14, 1e-10)
plt.grid(which="both")

In [ ]:
yaxis_units="erg cm-2 s-1"
ax = plt.subplot()
ax.yaxis.set_units(u.Unit(yaxis_units))
ax.xaxis.set_units(u.Unit("TeV"))

flux_points = FluxPoints.from_table(table = table_HAWCJ1825_134, sed_type='e2dnde')
flux_points.plot(ax=ax)


plt.xlim(.1, 1000)
plt.ylim(1e-14, 1e-10)
plt.grid(which="both")
plt.savefig('uu.png', bbox_inches='tight')

In [ ]:
flux_points.to_table()

In [ ]:
table_HAWCJ1825_134["e2dnde"][:] = table_HAWCJ1825_134["e2dnde"].to(u.Unit("erg cm-2 s-1")) 

In [ ]:
table_HAWCJ1825_134

In [ ]:
flux_points = FluxPoints.from_table(table_HAWCJ1825_134, sed_type='e2dnde')

flux_points.to_table(formatted=True)

#### LHAASO J1825_KM2A

In [ ]:
from pathlib import Path
directory_path = '../publishNature'
format_dat = '.dat'
file_name = "J1825_KM2A_201209"
file_abs_name = f'{file_name}{format_dat}'

file_path = Path(f'{directory_path}/{file_abs_name}') 
J1825 = Table.read(file_path,format='ascii', delimiter=' ', comment='#')
J1825

In [ ]:
from astropy.table import QTable
import astropy.units as u

J1825['col1'] = J1825['col1']/1e12
J1825.rename_column('col1', 'e_ref')
J1825['e_ref'].unit = u.TeV

J1825['col5'] = J1825['col5']/1e12
J1825.rename_column('col5', 'e_min')
J1825['e_min'].unit = u.TeV

J1825['col6'] = J1825['col6']/1e12
J1825.rename_column('col6', 'e_max')
J1825['e_max'].unit = u.TeV



J1825.rename_column('col2', 'e2dnde')
J1825['e2dnde'].unit = u.Unit("erg cm-2 s-1")

J1825.rename_column('col3', 'e2dnde_errp')
J1825['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")

J1825.rename_column('col4', 'e2dnde_errn')
J1825['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

J1825.meta["SED_TYPE"] = "e2dnde"
J1825.meta["name"] = "J1825"





# # Writes the flux points table in the csv format
# file_abs_name = f'{file_name}{format_fits}'
# path_os = os.path.abspath(
#     os.path.join(
#         f"{directory_path}/{file_abs_name}"
#     )
# )

# if path_os not in sys.path:
#     sys.path.append(path_os)

# J1825.write(
#     f"{path_os}",
#     format = 'fits', 
#     overwrite = True
# )   

# # Writes the flux points table in the csv format
# file_abs_name = f'{file_name}{format_csv}'
# path_os = os.path.abspath(
#     os.path.join(
#         f"{directory_path}/{file_abs_name}"
#     )
# )

# if path_os not in sys.path:
#     sys.path.append(path_os)

# J1825.write(
#     f"{path_os}",
#     format = 'ascii.ecsv', 
#     overwrite = True
# )  

In [ ]:

J1825

In [ ]:
FluxPoints.from_table(table = J1825, sed_type='e2dnde').plot()

In [ ]:
J1825['e_ref'],J1825['e2dnde']

In [ ]:
x_data = J1825['e_ref']
y_data = J1825['e2dnde']

In [ ]:
def power_law(energy,index, amplitude):  
    return amplitude*(energy)**(-index)

In [ ]:
from scipy.optimize import curve_fit
popt, pcov = curve_fit(power_law, J1825['e_ref'], J1825['e2dnde'], p0=[2,1e-12])

In [ ]:
popt, pcov

In [ ]:
a_opt, b_opt = popt
x_plw = np.linspace(min(x_data), max(x_data), 100)
y_plw = power_law(x_plw, a_opt, b_opt)

In [ ]:
def log_parabola(energy, amplitude, alpha, beta):  
    return amplitude*((energy/10)**(-(-alpha+(beta*np.log(energy/10)))))

In [ ]:
popt, pcov = curve_fit(log_parabola, J1825['e_ref'], J1825['e2dnde'], p0=[1e-12,1, 1])

In [ ]:
popt, pcov

In [ ]:
a_opt, b_opt, c_opt = popt
x_lp = np.linspace(min(x_data), max(x_data), 100)
y_lp = log_parabola(x_lp, a_opt, b_opt, c_opt)

In [ ]:
energy_bounds=[10, 1000]* u.TeV

pwl = PowerLawSpectralModel(
    index=3.36, 
    amplitude="1e-12 cm-2 s-1 erg-1", 
    reference="1 TeV"
)


plt.figure(figsize=(4,6))
plt.loglog(J1825['e_ref'],J1825['e2dnde'], 'o', color='red', lw=2, ms=4)
plt.errorbar(J1825['e_ref'],J1825['e2dnde'], yerr=[J1825['e2dnde_errn'], J1825['e2dnde_errp']],fmt='.', color='red')



# pwl.plot(energy_bounds=energy_bounds, color="k")
plt.text(0.04,0.1, PeVatron_name, transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black'), size=13)

plt.loglog(x_plw,y_plw, '--' ,color='black')
# plt.loglog(x_lp,y_lp, '--' ,color='black')

plt.xlabel("Energy (TeV)")   
plt.ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
plt.xlim(10, 1000)
plt.ylim(1e-14, 5e-11)
plt.grid(which="both")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12,6))
ax = axes[0]
ax.loglog(J1825['e_ref'],J1825['e2dnde'], 'o', color='red', lw=2, ms=4)
ax.errorbar(J1825['e_ref'],J1825['e2dnde'], yerr=[J1825['e2dnde_errn'], J1825['e2dnde_errp']],fmt='.', color='red')
ax.text(0.04,0.1, PeVatron_name, transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black'), size=13)
ax.loglog(x_plw,y_plw, '--' ,color='black')
ax.set_ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
ax.grid(which="both")





fig.text(0.5, -0.04, "Energy (TeV)", ha='center', size=20)

# plt.ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
plt.xlim(10, 1000)
plt.ylim(1e-14, 5e-11)
plt.grid(which="both")



In [ ]:
from pathlib import Path
from astropy.table import QTable
import astropy.units as u
from scipy.optimize import curve_fit


In [ ]:
def power_law(energy,index, amplitude):  
    return amplitude*(energy)**(-index)

In [ ]:
def log_parabola(energy, amplitude, alpha, beta):  
    return amplitude*((energy/10)**(-(-alpha+(beta*np.log(energy/10)))))

In [ ]:
file_path

In [ ]:
for i, file_name in enumerate(files_names):
    PeVatron_name = PeVatrons_names[i]
    print(PeVatron_name)
    file_abs_name = f'{file_name}'

    file_path = Path(f'{directory_path}/{file_abs_name}') 
    table = Table.read(file_path,format='ascii', delimiter=' ', comment='#')
    display(table)
    
    table['col1'] = table['col1']/1e12
    table.rename_column('col1', 'e_ref')
    table['e_ref'].unit = u.TeV

    table['col5'] = table['col5']/1e12
    table.rename_column('col5', 'e_min')
    table['e_min'].unit = u.TeV

    table['col6'] = table['col6']/1e12
    table.rename_column('col6', 'e_max')
    table['e_max'].unit = u.TeV

    table.rename_column('col2', 'e2dnde')
    table['e2dnde'].unit = u.Unit("erg cm-2 s-1")

    table.rename_column('col3', 'e2dnde_errp')
    table['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")

    table.rename_column('col4', 'e2dnde_errn')
    table['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

    table.meta["SED_TYPE"] = "e2dnde"
    table.meta["name"] = "table"
    display(table)
    x_data = table['e_ref']
    y_data = table['e2dnde']

    popt, pcov = curve_fit(power_law, table['e_ref'], table['e2dnde'], p0=[2,1e-12])
    a_opt, b_opt = popt
    x_plw = np.linspace(min(x_data), max(x_data), 100)
    y_plw = power_law(x_plw, a_opt, b_opt)
    fig, axes = plt.subplots(1, 3, figsize=(12,6))
    ax = axes[0]
    ax.loglog(table['e_ref'], table['e2dnde'], 'o', color='red', lw=2, ms=4)
    ax.errorbar(table['e_ref'], table['e2dnde'], yerr=[table['e2dnde_errn'], table['e2dnde_errp']],fmt='.', color='red')
    ax.text(0.04,0.1, PeVatron_name, transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black'), size=13)
    ax.loglog(x_plw,y_plw, '--' ,color='black')
    ax.set_ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
    ax.grid(which="both")

    fig.text(0.5, -0.04, "Energy (TeV)", ha='center', size=20)

    plt.xlim(10, 1000)
    plt.ylim(1e-14, 5e-11)
    plt.grid(which="both")
    plt.show()


In [ ]:
file_name = files_names[0]
file_abs_name = f'{file_name}'

file_path = Path(f'{directory_path}/{file_abs_name}') 
table = Table.read(file_path,format='ascii', delimiter=' ', comment='#')
table

In [ ]:
table['col1'] = table['col1']/1e12
table.rename_column('col1', 'e_ref')
table['e_ref'].unit = u.TeV

table['col5'] = table['col5']/1e12
table.rename_column('col5', 'e_min')
table['e_min'].unit = u.TeV

table['col6'] = table['col6']/1e12
table.rename_column('col6', 'e_max')
table['e_max'].unit = u.TeV

table.rename_column('col2', 'e2dnde')
table['e2dnde'].unit = u.Unit("erg cm-2 s-1")

table.rename_column('col3', 'e2dnde_errp')
table['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")

table.rename_column('col4', 'e2dnde_errn')
table['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

table.meta["SED_TYPE"] = "e2dnde"
table.meta["name"] = "table"

In [ ]:
table

In [ ]:
x_data = table['e_ref']
y_data = table['e2dnde']

Power law

In [ ]:
popt, pcov = curve_fit(power_law, J1825['e_ref'], J1825['e2dnde'], p0=[2,1e-12])
a_opt, b_opt = popt
x_plw = np.linspace(min(x_data), max(x_data), 100)
y_plw = power_law(x_plw, a_opt, b_opt)

In [ ]:
# fig, axes = plt.subplots(1, 3, figsize=(12,6))
# ax = axes[0]
# ax.loglog(table['e_ref'],table['e2dnde'], 'o', color='red', lw=2, ms=4)
# ax.errorbar(table['e_ref'],table['e2dnde'], yerr=[J1825['e2dnde_errn'], J1825['e2dnde_errp']],fmt='.', color='red')
# ax.text(0.04,0.1, PeVatron_name, transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black'), size=13)
# ax.loglog(x_plw,y_plw, '--' ,color='black')
# ax.set_ylabel(r"$E^{2}$dN/dE (erg $cm^{-2}$ $s^{-1}$)")
# ax.grid(which="both")

# fig.text(0.5, -0.04, "Energy (TeV)", ha='center', size=20)

# plt.xlim(10, 1000)
# plt.ylim(1e-14, 5e-11)
# plt.grid(which="both")

In [ ]:
ax = plt.subplot()
ax.loglog(J1825['e_ref'],J1825['e2dnde'], 'o', color='red', lw=2, ms=4)
ax.errorbar(J1825['e_ref'],J1825['e2dnde'], yerr=[J1825['e2dnde_errn'], J1825['e2dnde_errp']],fmt='.', color='red')



In [ ]:
plt.loglog(J1825['e_ref'],J1825['e2dnde'], 'o', color='red', lw=2, ms=4)
plt.errorbar(J1825['e_ref'],J1825['e2dnde'], yerr=[J1825['e2dnde_errn'], J1825['e2dnde_errp']],fmt='.', color='red')


In [ ]:
# file_name = "J1825_KM2A_201209"
# file_abs_name = f'{file_name}{format_fits}'

# file_path = Path(f'{directory_path}/{file_abs_name}') 
# J1825_t = Table.read(file_path,format='fits')
# J1825_t

In [ ]:
J1825.meta

In [ ]:
FluxPoints.from_table(table = J1825, sed_type='e2dnde')

In [ ]:
file_path = Path(f'../publishNature/J1908_KM2A_201209.dat') 
J1908 = Table.read(file_path ,format='ascii', delimiter=' ', comment='#')
J1908

In [ ]:
file_path = Path(f'../publishNature/J2228_KM2A_201209.dat') 
J2228 = Table.read(file_path,format='ascii', delimiter=' ', comment='#')
J2228

In [ ]:
J1825

In [ ]:
flux_points_ = FluxPoints.from_table(J1825, sed_type="e2dnde")

In [ ]:
flux_points_.plot()

In [ ]:
# e_ref = J1825['col1']/1e12
# e_min = J1825['col5']/1e12
# e_max = J1825['col6']/1e12

# e2dnde = J1825['col2']
# e2dnde_errp = J1825['col3']
# e2dnde_errn = J1825['col4']

# J1825_table = Table(
#     [e_ref, e2dnde, e2dnde_errp, e2dnde_errn, e_min, e_max],
#     names=('e_ref', 'e2dnde', 'e2dnde_errp', 'e2dnde_errn', 'e_min', 'e_max'),
#     meta={'name': 'J1825','SED_TYPE': 'e2dnde' }
# )

# J1825_table['e_ref'].unit = u.TeV
# J1825_table['e_min'].unit = u.TeV
# J1825_table['e_max'].unit = u.TeV

# J1825_table['e2dnde'].unit = u.Unit("erg cm-2 s-1")
# J1825_table['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")
# J1825_table['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

In [ ]:
table_J1825.meta

In [ ]:
flux_points2 = FluxPoints.from_table(table_J1825, sed_type="e2dnde")
flux_points2.to_table()

In [ ]:
e_ref = J1908['col1']
e2dnde = J1908['col2']
e2dnde_errp = J1908['col2']
e2dnde_errn = J1908['col3']
J1908_table = QTable([e_ref, e2dnde, e2dnde_errp, e2dnde_errn],
           names=('e_ref', 'e2dnde', 'e2dnde_errp', 'e2dnde_errn'),
           meta={'name': 'J1908'})

J1908_table['e_ref'] = J1908['col1']/1e12
J1908_table['e_ref'].unit = u.TeV
J1908_table['e2dnde'].unit = u.Unit("erg cm-2 s-1")
J1908_table['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")
J1908_table['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

In [ ]:
J1908_table

In [ ]:
e_ref = J2228['col1']
e2dnde = J2228['col2']
e2dnde_errp = J2228['col3']
e2dnde_errn = J2228['col4']
J2228_table = QTable([e_ref, e2dnde, e2dnde_errp, e2dnde_errn],
           names=('e_ref', 'e2dnde', 'e2dnde_errp', 'e2dnde_errn'),
           meta={'name': 'J2228'})

J2228_table['e_ref'] = J2228['col1']/1e12
J2228_table['e_ref'].unit = u.TeV
J2228_table['e2dnde'].unit = u.Unit("erg cm-2 s-1")
J2228_table['e2dnde_errp'].unit = u.Unit("erg cm-2 s-1")
J2228_table['e2dnde_errn'].unit = u.Unit("erg cm-2 s-1")

In [ ]:
flux_points_ = FluxPoints.from_table(table_J1825)
flux_points_.to_table()

In [ ]:
flux_points_.plot()

In [ ]:
flux_points_.to_table()

In [ ]:
# ds = FluxPointsDataset(
#     data = flux_points_, 
#     name = f"{counterpart_name}: {catalog_tag}"
# )
# datasets = Datasets(ds)


# # opts = dict(sed_type="e2dnde", yunits=u.Unit("TeV cm-2 s-1"))
# energy_bounds = [10, 1e3] * u.TeV
# ylim = [1e-14, 3e-11]
# plotting_SED_flux_pointing(
#     datasets = datasets,
# #     spectral_model=spectral_model, 
#     sed_type = "e2dnde", 
#     yaxis_units="erg cm-2 s-1", 
#     energy_bounds=energy_bounds, 
#     ylim=ylim)


In [ ]:
# table.rename_column('col2', 'e_ref')  
# table["e_ref"].unit = u.eV
# table["e_ref"]

In [ ]:
flux_points_.plot()

#### Let's plot the spectrum

    TF1 *f=new TF1("f","[0]*pow(x/20.0,[1])*pow(x,2)",10,2000);
          f->SetParameters(1.5e-11,-2.4);

      TF1 *f2=new TF1("f2","[0]*pow(x/20.,[1]+[2]*log10(x/20.))*pow(x,2)");
          f2->SetParameters(2.e-14,-2.6,-0.2,1000);

In [ ]:
catalog = CATALOG_REGISTRY.get_cls(catalog_tag)()
source  = catalog[counterpart_name]
# Gammapy sky model object:
sky_model = source.sky_model()

# Gammapy spatial model object:
spatial_model = sky_model.spatial_model

# Gammapy spectral model object:
spectral_model = sky_model.spectral_model
spectral_model_tag = spectral_model.tag[0]

In [ ]:
source.data

In [ ]:
print(sky_model)

In [ ]:
print(spatial_model)

In [ ]:
print(spectral_model)

In [ ]:
cat_index = 0
datasets_fp[source_index][cat_index]

In [ ]:
def plot_spectrum(data = None):
    plt.style.use(['science', 'notebook', 'grid'])
    fig, ax = plt.subplots(1, 1, figsize=(6,5))

    kwargs_model = {
        "color":"black",
        "ls":"--"
    }
    kwargs_fp = {
        "color":"blue", 
        "marker":"o"
    }

    data = data
    data.plot_spectrum(
        ax = ax,  
        kwargs_fp = kwargs_fp, 
        kwargs_model = kwargs_model
    )

    title = f"{PeVatron_name}: {counterpart_name}\n{catalog_tag}: {spectral_model_tag}"

    ax.text(
        0.1, 0.1, 
        title,
        transform = ax.transAxes,
        bbox = dict(
                facecolor = 'white', 
                edgecolor = 'black'
        ), 
        size = 10
    )
    ax.tick_params(axis='both', which='both', labelsize=10)

    file = path_plot_spectrum / f'{title}.png'

    plt.savefig(file, bbox_inches='tight')
    plt.show()

In [ ]:
data = datasets_fp[source_index][cat_index]
plot_spectrum(data)

In [ ]:
# kwargs = {
#         "ax": ax, 
# #         "sed_type": sed_type
#     }    
# kwargs_fit = {"label": spectral_model_tag}
# spec_model.plot(energy_bounds=energy_bounds, color="k", **kwargs, **kwargs_fit)
# spec_model.plot_error(energy_bounds=energy_bounds, **kwargs)

# plt.show()


In [ ]:
def plot_flux_points(spectral_model, ax, label, color):
    
    energy_bounds = (100 * u.MeV, 100 * u.GeV)
    spectral_model.plot(
        ax=ax, label=label, color=color
    )
    spec.plot_error(ax=ax, energy_bounds=energy_bounds, color=color)


In [ ]:
plt.style.use(['science', 'notebook', 'grid'])

def plot_spectrum(spectral_model, ax, label, color):
    
    energy_bounds = (100 * u.MeV, 100 * u.GeV)
    spectral_model.plot(
        ax=ax, energy_bounds=energy_bounds,label=label, color=color
    )
    spectral_model.plot_error(ax=ax, energy_bounds=energy_bounds, color=color)

fig, ax = plt.subplots()
plot_spectrum(spectral_model, ax=ax, label=spectral_model_tag, color="tab:blue")
# dataset.data.plot(label=d.name, **kwargs)
#plot_spectrum(model_joint, ax=ax, label="joint", color="tab:orange")
ax.legend()
plt.show()

In [ ]:
dataset_fp = FluxPointsDataset(data=source.flux_points,models=sky_model, name=spectral_model_tag)
dataset_fp.data.to_table(sed_type=sed_type, formatted=True)



In [ ]:

datasets = Datasets([dataset_fp])
datasets.models = sky_model
print(datasets)

fitter = Fit()
result_pwl = fitter.run(datasets=datasets)

In [ ]:
plt.style.use(['default'])

ax = plt.subplot()
ax.yaxis.set_units(u.Unit("erg cm-2 s-1"))

kwargs = {"ax": ax, "sed_type": "e2dnde"}

for d in datasets:
    d.data.plot(label=d.name, **kwargs)

# energy_bounds = (100 * u.MeV, 100 * u.GeV)
spectral_model.plot(energy_bounds=energy_bounds, color="k", **kwargs)
# ax.set_ylim(1e-13, 1e-8)
ax.set_xlim(energy_bounds)
ax.legend()
plt.show()

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(12,4))

# ax.plot(x, y, 'o--', color='r', lw=0.4, ms=3)


# ax.text(0.1, 0.1, 'text here', transform=ax.transAxes)
# ax.set_xlabel('Time [s]')
# ax.set_ylabel(r'$\frac{d}{dx} f(x)$', fontsize=15)
# plt.show()


In [ ]:
# #load precomputed flux points
# flux_points = source.flux_points
# model = SkyModel(spectral_model=spectral_model)
# dataset = FluxPointsDataset(model, flux_points)
# dataset.plot_spectrum()

In [ ]:
# #load precomputed flux points
# flux_points = source.flux_points
# model = SkyModel(spectral_model=spectral_model)
# dataset = FluxPointsDataset(model, flux_points)
# #configuring optional parameters
# kwargs_spectrum = {"kwargs_model": {"color":"red", "ls":"--"}, "kwargs_fp":{"color":"green", "marker":"o"}}  # noqa: E501
# kwargs_residuals = {"color": "blue", "markersize":4, "marker":'s', }
# dataset.plot_fit(kwargs_residuals=kwargs_residuals, kwargs_spectrum=kwargs_spectrum) 


In [ ]:
# plt.style.use(['science', 'notebook', 'grid'])


# fig, axes = plt.subplots(1, 2, figsize=(10,3.5))
# data = datasets_fp[source_index][cat_index]
# data.plot_spectrum(kwargs_fp=kwargs_fp, kwargs_model=kwargs_model)
# ax = axes[0]
# ax.text(0.1, 0.1, 'text here', transform=ax.transAxes)







# fig, ax = plt.subplots(1, 1, figsize=(5,4))
# data = datasets_fp[source_index][cat_index]
# data.plot_residuals()
# ax = axes[1]
# ax.text(0.1, 0.1, 'text here', transform=ax.transAxes)
# # ax.set_xlabel('Time [s]')
# # ax.set_ylabel(r'$\frac{d}{dx} f(x)$', fontsize=15)
# plt.show()


In [ ]:
# plt.style.use(['science', 'notebook', 'grid'])


# fig, ax = plt.subplots(1, 1, figsize=(5,4))
# data = datasets_fp[source_index][cat_index]
# data.plot_residuals()
# ax.text(0.1, 0.1, 'text here', transform=ax.transAxes)
# # ax.set_xlabel('Time [s]')
# # ax.set_ylabel(r'$\frac{d}{dx} f(x)$', fontsize=15)
# plt.show()

In [ ]:
# energy_bounds = (100 * u.MeV, 100 * u.GeV)

# flux_points.plot()

# spectral_model.plot(energy_bounds=energy_bounds)
# # ax.set_yscale('linear')


In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10,3.5))
# ax = axes[0]

# data = datasets_fp[source_index][cat_index]
# ax.plot_spectrum(data=data, kwargs_fp=kwargs_fp, kwargs_model=kwargs_model)

# ax.text(0.05, 0.81, textstr1, transform=ax.transAxes,
#        bbox=dict(facecolor='white', edgecolor='black'), size=12)

# ax.legend(fontsize=10, fancybox=False, edgecolor='black')
# ax.set_ylabel('Frequency')
# ax.set_title('Trial 1')
# ax = axes[1]
# ax.hist(res_a2, bins=30, density=True, histtype='step', label='Method 1', color='blue')
# ax.hist(res_b2, bins=30, density=True, histtype='step', label='Method 2', color='red')
# ax.text(0.05, 0.81, textstr2, transform=ax.transAxes,
#        bbox=dict(facecolor='white', edgecolor='black'), size=12)
# ax.set_title('Trial 2')
# fig.text(0.5, -0.04, '$\Delta E$ [Joules]', ha='center', size=20)
# plt.savefig('trial.png', dpi=200)
# plt.show()

In [ ]:
# file_name = title.replace(" ", "").replace(":", "_").replace("\n", "_")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,6))

#configuring optional parameters
kwargs_model = {"color":"black", "ls":"--"}
kwargs_fp = {"color":"blue", "marker":"o"}
data = datasets_fp[source_index][cat_index]
data.plot_residuals(ax = ax)
ax.text(0.1, 0.1, f"spectral model: {spectral_model_tag}\ncatalog: {catalog_tag}", transform=ax.transAxes)
ax.legend()

# ax.set_xlabel('Time [s]')
# ax.set_ylabel(r'$\frac{d}{dx} f(x)$', fontsize=15)

plt.title(f"{PeVatron_name}: {counterpart_name}")
# plt.grid(True)

In [ ]:
plt.style.use(['science', 'notebook', 'grid'])
#plt.figure(figsize=(5,4))


#configuring optional parameters
kwargs_model = {"color":"black", "ls":"--"}
kwargs_fp = {"color":"blue", "marker":"o"}
data = datasets_fp[source_index][cat_index]
data.plot_spectrum(kwargs_fp=kwargs_fp, kwargs_model=kwargs_model)
# fig = plt.figure()
# fig.set_label('s')
plt.title(f"{PeVatron_name}: {counterpart_name}")
plt.grid(True)

In [ ]:
print(datasets_fp[source_index][cat_index].plot_fit)

In [ ]:
datasets_fp[source_index][cat_index].plot_fit()

In [ ]:
datasets_fp[source_index][cat_index].plot_residuals()

In [ ]:
from itertools import combinations
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
from IPython.display import display
from gammapy.datasets import Datasets, SpectrumDatasetOnOff
from gammapy.modeling import Fit
from gammapy.modeling.models import LogParabolaSpectralModel, SkyModel

In [ ]:
crab_spectrum = LogParabolaSpectralModel(
    amplitude=1e-11 / u.cm**2 / u.s / u.TeV,
    reference=1 * u.TeV,
    alpha=2.3,
    beta=0.2,
)

crab_spectrum.alpha.max = 3
crab_spectrum.alpha.min = 1
crab_model = SkyModel(spectral_model=crab_spectrum, name="crab")

In [ ]:
datasets = []
for obs_id in [23523, 23526]:
    dataset = SpectrumDatasetOnOff.read(
        f"$GAMMAPY_DATA/joint-crab/spectra/hess/pha_obs{obs_id}.fits"
    )
    datasets.append(dataset)

dataset_hess = Datasets(datasets).stack_reduce(name="HESS")
datasets = Datasets(datasets=[dataset_hess])

# Set model and fit range
dataset_hess.models = crab_model
e_min = 0.66 * u.TeV
e_max = 30 * u.TeV
dataset_hess.mask_fit = dataset_hess.counts.geom.energy_mask(e_min, e_max)

In [ ]:
scipy_opts = {
    "method": "L-BFGS-B",
    "options": {"ftol": 1e-4, "gtol": 1e-05},
    "backend": "scipy",
}
fit_scipy = Fit(store_trace=True, optimize_opts=scipy_opts)

In [ ]:
result_scipy = fit_scipy.run(datasets)

In [ ]:
sherpa_opts = {"method": "simplex", "ftol": 1e-3, "maxfev": int(1e4)}
fit_sherpa = Fit(store_trace=True, backend="sherpa", optimize_opts=sherpa_opts)
results_simplex = fit_sherpa.run(datasets)

In [ ]:
fit = Fit(store_trace=True)
minuit_opts = {"tol": 0.001, "strategy": 1}
fit.backend = "minuit"
fit.optimize_opts = minuit_opts
result_minuit = fit.run(datasets)

In [ ]:
plt.figure()
energy_bounds = [1, 10] * u.TeV
crab_spectrum.plot(energy_bounds=energy_bounds, energy_power=2)
ax = crab_spectrum.plot_error(energy_bounds=energy_bounds, energy_power=2)

In [ ]:
from gammapy.modeling import Fit
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel
from gammapy.estimators import FluxPoints
from gammapy.datasets import FluxPointsDataset
filename = "$GAMMAPY_DATA/tests/spectrum/flux_points/diff_flux_points.fits"
dataset = FluxPointsDataset.read(filename)
model = SkyModel(spectral_model=PowerLawSpectralModel())
dataset.models = model

In [ ]:
fit = Fit()
result = fit.run([dataset])
print(result)

In [ ]:
print(result.parameters.to_table())